In [2]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
from selenium.webdriver.common.by import By
import pandas as pd
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 셀레니움 신버전 크롬 드라이버 패키지

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
# 진행률
from tqdm.notebook import tqdm
import re

In [4]:
연동_data_df=pd.read_csv('제주도식당/제주시/연동/id/연동_id_name.csv' , encoding= 'euc-kr')

In [6]:
연동_data_df

,사업장명,id
0,봉플라봉뱅,1713847517
1,스톤아일랜드탭하우스,1244422132
2,연화양꼬치,1482063535
3,24시누름돌김치찌개,1028483996
4,영화루,1109507832
...,...,...
1014,토라네코제주본점,1778945379
1015,지에스25연동가락점,1326118478
1016,귀아랑2길베이커리카페,1887309829
1017,세븐일레븐제주연동노연점,1255085555


In [8]:
tst = 연동_data_df[:5]
tst

,사업장명,id
0,봉플라봉뱅,1713847517
1,스톤아일랜드탭하우스,1244422132
2,연화양꼬치,1482063535
3,24시누름돌김치찌개,1028483996
4,영화루,1109507832


In [14]:
tst.loc[:, 'samp'] = 'hellow'

C:\Users\Admin\AppData\Local\Temp\ipykernel_6876\845351278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tst.loc[:, 'samp'] = 'hellow'


In [17]:
tst

,사업장명,id,samp
0,봉플라봉뱅,1713847517,hellow
1,스톤아일랜드탭하우스,1244422132,hellow
2,연화양꼬치,1482063535,hellow
3,24시누름돌김치찌개,1028483996,hellow
4,영화루,1109507832,hellow


In [1]:
# 문장 분할 X
def naver_rest_review(rest_name , rest_id) :    
    
    name = rest_name
    id_num = rest_id
    
    # Chrome headless 설정
    options = Options()
    # options.add_argument('--headless')

    # 드라이버 생성
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # name = '고기다 성산흑돼지'
    # 리뷰 페이지 열기
    print(f'현재 크롤링이 진행중인 검색어 : {name}')

    url = f"https://map.naver.com/v5/search/{name}/place/{id_num}"

    driver.get(url)
    wait = WebDriverWait(driver, 5)
    iframe = wait.until(EC.presence_of_element_located((By.ID, "entryIframe")))
    driver.switch_to.frame(iframe)

    # 리뷰 클릭
    # veBoZ class tag에 해당하는 요소가 존재하는지 확인
    try :
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "veBoZ")))
        category = driver.find_elements(by = By.CLASS_NAME , value = 'veBoZ')
        tlst = [c.text for c in category]
        if '리뷰' not in tlst :
            telst = ['no_review']
            df = pd.DataFrame(telst , columns = ['리뷰'])
            df.to_csv(f'제주도식당/제주시/연동/리뷰/{name}_review.csv' , index = False , encoding='euc-kr')

            return telst
        else :
            for num , tag in enumerate(category) :
                # print(tag.text)
                if tag.text == '리뷰' :
                    driver.find_elements(by = By.CLASS_NAME
                                        , value = 'veBoZ')[num].click()
    except :
        pass

    # 최신순 클릭
    # mSdTM class tag가 선택 가능한지 확인
    
    try :
        time.sleep(1.5)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "mSdTM")))
        # wait.until(EC.element_to_be_clickable((By.CLASS_NAME , "mSdTM")))
        driver.find_elements(by = By.CLASS_NAME , value = "mSdTM")[1].click()
    except :
        pass
    

    # 리뷰 전체 펼치기
    try :
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME , "fvwqf")))
        click = driver.find_element(by = By.CLASS_NAME , value = 'fvwqf')

        while click is not None:
            try :
                click.click()  
            except :
                break
    except :
        print('펼칠 리뷰가 없다.')
        pass

    # driver 객체를 사용하여 body 태그를 찾고, Keys.HOME을 사용하여 스크롤을 맨 위로 이동합니다.
    # fvwqf class tag가 보이지 않으면 진행합니다.
    try :
        wait.until(EC.invisibility_of_element_located((By.CLASS_NAME , "fvwqf")))
    except :
        pass
    body = driver.find_element(by = By.TAG_NAME , value = 'body')
    body.send_keys(Keys.HOME)
    time.sleep(0.4)
    review_click = driver.find_elements(by = By.CLASS_NAME , value = 'rvCSr')
    if len(review_click) != 0 :
        for address in tqdm(review_click) :
            address.click()
            time.sleep(0.2)
    else :
        pass

    # 리뷰 가져오기
    # 현재는 리뷰만 가져오지만 가져올 데이터를 어떤 것을 정하느냐에 따라 더 많은 정보를 가져올 수도 있다.

    review_lst = []

    review = driver.find_elements(by = By.CLASS_NAME , value = 'zPfVt')

    for rev in tqdm(review) :
        review_lst.append(rev.text)

    telst = []
    for review in tqdm(review_lst) :
        split_result = [review_s.split('. ') for review_s in review.split('\n')]
        for text_1 in split_result :
            for text_2 in text_1 :
                # 특수문자 알파벳 단일 자음,모음 제거
                pattern = r"[^가-힣\s]+"
                result = re.sub(pattern, "", text_2)
                # 추가로 2칸 이상 공백 1칸으로 통일
                result = re.sub(r"[\s{2,}]", " ", result)
                telst.append(result)        
    # df = pd.DataFrame(telst , columns = ['리뷰'])
    df = pd.DataFrame(review_lst , columns = ['리뷰'])
    df['id'] = rest_id
    df.to_excel(f'제주도식당/제주시/연동/리뷰2/{name}_review.xlsx' , index = False)
    driver.quit()
    
    return telst

In [ ]:
rena = 연동_data_df.사업장명.tolist()
reid = 연동_data_df.id.tolist()

In [ ]:
for name , id_r in zip(rena , reid) :
    print(name , id_r)
    globals()[f'{name}_review_lst'] = naver_rest_review(name , id_r)
